# 불필요 요소

In [19]:
from urllib.request import urlopen
import requests
import json

# URL 문자열

domain = "https://apihub.kma.go.kr/api/typ01/url/fct_shrt_reg.php?"
reg_id = "reg=11B00000&"
tm_st = "tm_st=2021-07-01&"
tm_ed = "tm_ed=2021-07-02&"
reg_sp = "A"
reg_name = ""
option = "authKey="
auth = "I4adfRxGTwmGnX0cRp8JgA"

url = domain + reg_id + tm_st + tm_ed + reg_sp + reg_name + option + auth

# GET 요청
# response = requests.get(url)
with urlopen(url) as r:
    html = r.read().decode('cp949')
    print(html)

HTTPError: HTTP Error 401: Unauthorized

In [ ]:
# url3 = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={serviceKeyE}&pageNo=1&numOfRows=1000&dataType=XML&base_date=20250325&base_time=0500&nx=55&ny=127"

# print(response.text)

# req = Request(url = url3 , headers={'User-Agent' :  generate_user_agent()})
# response = urlopen(req, context=context)

# response

# response = requests.get(url3)
# print(response.content)

# context = ssl.create_default_context()
# context.set_ciphers('DEFAULT@SECLEVEL=1')



#------------------JSON load 결과-----------------------
# response = urlopen(req)


#-------------------디코드 결과-------------------------
# response.read().decode('cp949')

#---------------------JOSN 결과------------------------
# res = response.json()
# print(res)

import pandas as pd

# 데이터 예시 (실제로는 API에서 가져온 데이터를 사용할 것)
data = [
    {"category": "TMP", "name": "1시간 기온", "val": "9", "date": "20250326", "time": "0900"},
    {"category": "UUU", "name": "동서바람", "val": "-1.3", "date": "20250326", "time": "0900"},
    # ... 나머지 데이터
]

# 데이터프레임 생성
df = pd.DataFrame(data)

# 날짜와 시간을 합쳐서 datetime 인덱스 만들기
df['datetime'] = pd.to_datetime(df['date'] + df['time'], format='%Y%m%d%H%M')

# 피벗 테이블로 변환 - 날짜시간을 인덱스로, 카테고리를 컬럼으로
pivot_df = df.pivot(index='datetime', columns='category', values='val')

# 필요하면 숫자형으로 변환
for col in pivot_df.columns:
    try:
        pivot_df[col] = pd.to_numeric(pivot_df[col])
    except:
        pass  # 숫자로 변환할 수 없는 경우 패스

print(pivot_df)

# 예상 결과....

datetime           TMP   UUU   VEC   ...
2025-03-26 09:00    9   -1.3  -0.7   ...
2025-03-26 10:00   ...   ...   ...   ...

최근 발표 날짜: 20250327, 시간: 1100


# 단기예보 API 구동

In [81]:
import ssl
import pandas as pd
from datetime import datetime, timedelta
import json
import requests

#---------------------------------가장 최신 발표 시간 얻는 함수-----------------------------------------
def get_latest_forecast_time():
    # 발표 시간 목록 (24시간 형식)
    forecast_times = ["0200", "0500", "0800", "1100", "1400", "1700", "2000", "2300"]
    
    # 현재 시간
    now = datetime.now()
    
    # 현재 날짜 문자열 (YYYYMMDD)
    current_date = now.strftime("%Y%m%d")
    
    # 현재 시간 문자열 (HHMM)
    current_time = now.strftime("%H%M")
    
    # 오늘 발표된 시간 중 현재 시간보다 이전인 것 찾기
    available_times = [t for t in forecast_times if t <= current_time]
    
    # 오늘 발표된 시간 중 가장 최근 시간
    if available_times:
        latest_time = max(available_times)
        return current_date, latest_time
    else:
        # 오늘 발표된 시간이 없는 경우 (현재 시간이 0200 이전)
        # 전날 마지막 발표 시간 사용
        yesterday = now - timedelta(days=1)
        yesterday_date = yesterday.strftime("%Y%m%d")
        return yesterday_date, forecast_times[-1]  # 2300

#-----------------------------------API 호출------------------------------------------

# 날씨를 알고 싶은 시간 입력
date, time = get_latest_forecast_time()
nx = '62' # 예보 지점 x좌표
ny = '123' # 예보 지점 y좌표
numOfRows = 1000

url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0"
url2 = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"

serviceKeyE = "vf5wY%2FHyYSFgTBL63CIUXrE9QpdzM4Dklyhr%2FV3E0Zj9D7wMZSeiBfcr%2B27XEebGhg1RZAjdvDk9Rp2bF8PMwg%3D%3D"
serviceKeyD = "vf5wY/HyYSFgTBL63CIUXrE9QpdzM4Dklyhr/V3E0Zj9D7wMZSeiBfcr+27XEebGhg1RZAjdvDk9Rp2bF8PMwg=="


params = {
    'ServiceKey': serviceKeyD,
    'pageNo': '1',
    'numOfRows': numOfRows,
    'dataType': 'JSON',
    'base_date': date,
    'base_time': time,
    'nx': nx,
    'ny': ny,
}

responses = requests.get(url2, params=params)
response = json.loads(responses.text)


#-----------------------------------API 결과물들 List 로 저장------------------------------------------

section = response['response']['body']['items']['item']
dates = []
times = []
cates = []
values =[]

for i, row in enumerate(section):
    dates.append(row['fcstDate'])
    times.append(row['fcstTime'][:-2])
    cates.append(row['category'])
    values.append(row['fcstValue'])

#-----------------------------------df_api 원본 그대로 일단 표 생성------------------------------------------

df_api = pd.DataFrame({'일자':dates,
                        '시간': times,
                        '종류':cates,
                        '값':values})
df_api

#-----------------------------------Pivot_api 로 구조 정리------------------------------------------
pivot_api = df_api.pivot(index=['일자','시간'], columns='종류', values='값')

date_list = []
time_list = []
temp_list = []
rain_list = []
humid_list = []
wind_list = []
sunlight_list = []
radiation_list = []
snow_list = []

for i, row in enumerate(pivot_api.values):

    rain = pivot_api['PCP'].iloc[i]
    if rain == "강수없음":
        rain = 0
    elif rain == "1mm 미만":
        rain = 1
    elif rain == "2.0mm":
        rain = 2
    elif rain == "3.0mm":
        rain = 3

    snow = pivot_api['SNO'].iloc[i]
    if snow == "적설없음":
        snow = 0

    sky = pivot_api['SKY'].iloc[i]
    if sky == '1':  # 맑음
        sunlight = 1.0
        radiation = 3.0
    elif sky == '3':  # 구름 많음
        sunlight = 0.6
        radiation = 1.5
    elif sky == '4':  # 흐림
        sunlight = 0.2
        radiation = 0.5
    else:
        sunlight = 0.4  # 기본값
        radiation = 1.0


    date_list.append(pivot_api.index[i][0]) 
    time_list.append(pivot_api.index[i][1])
    temp_list.append(pivot_api['TMP'].iloc[i])
    rain_list.append(rain)
    humid_list.append(pivot_api['REH'].iloc[i])
    wind_list.append(pivot_api['WSD'].iloc[i])
    sunlight_list.append(sunlight)
    radiation_list.append(radiation)
    snow_list.append(snow)

#-----------------------------------df_fsct - 머신러닝에 맞는 구조로 변환------------------------------------------

df_fcst = pd.DataFrame({'일자': date_list,
                        '시간(시)': time_list,
                        '기온(℃)': temp_list, 
                        '강수량(mm)' : rain_list, 
                        '습도(%rh)' : humid_list, 
                        '풍속(m/s)' : wind_list, 
                        '일조(hr)' : sunlight_list, 
                        '일사(MJ/m2)': radiation_list,
                        '적설(cm)': snow_list})

df_fcst.head(72)

,일자,시간(시),기온(℃),강수량(mm),습도(%rh),풍속(m/s),일조(hr),일사(MJ/m2),적설(cm)
0,20250327,12,16,0,75,2.7,0.2,0.5,0
1,20250327,13,17,0,70,3.3,0.2,0.5,0
2,20250327,14,17,0,60,3.4,0.2,0.5,0
3,20250327,15,18,0,50,3.5,0.2,0.5,0
4,20250327,16,18,0,35,4.2,0.6,1.5,0
...,...,...,...,...,...,...,...,...,...
64,20250330,12,8,0,20,2,1.0,3.0,0
65,20250330,15,8,0,30,2,1.0,3.0,0
66,20250330,18,6,0,40,2,1.0,3.0,0
67,20250330,21,3,0,50,1,1.0,3.0,0


---
# 기존 완성된 코드들

In [39]:
weather_codes = {
    "POP": "강수확률  ",
    "PTY": "강수형태  ",
    "PCP": "1시간 강수량",
    "REH": "습도     ",
    "SNO": "1시간 신적설",
    "SKY": "하늘상태  ",
    "TMP": "1시간 기온",
    "TMN": "일 최저기온",
    "TMX": "일 최고기온",
    "UUU": "풍속(동서)",
    "VVV": "풍속(남북)",
    "WAV": "파고     ",
    "VEC": "풍향     ",
    "WSD": "풍속     "
}

for i in range(numOfRows):
    pt = res['response']['body']['items']['item'][i]
    date = pt['fcstDate']
    time = pt['fcstTime']
    cate = pt['category']
    val = pt['fcstValue']
    name = ''

    for code in weather_codes:
        if cate == code:
            name = weather_codes[code]
            break

    print(f"{cate}\t{name}\t={val}\t\tat {date} {time}시")

TMP	1시간 기온	=9		at 20250326 0900시
UUU	풍속(동서)	=-1.3		at 20250326 0900시
VVV	풍속(남북)	=0.7		at 20250326 0900시
VEC	풍향     	=117		at 20250326 0900시
WSD	풍속     	=1.6		at 20250326 0900시
SKY	하늘상태  	=3		at 20250326 0900시
PTY	강수형태  	=0		at 20250326 0900시
POP	강수확률  	=20		at 20250326 0900시
WAV	파고     	=-999		at 20250326 0900시
PCP	1시간 강수량	=강수없음		at 20250326 0900시
REH	습도     	=55		at 20250326 0900시
SNO	1시간 신적설	=적설없음		at 20250326 0900시


### (옵션)풍향 코드 계산

In [50]:
deg_code = {0 : 'N', 360 : 'N', 180 : 'S', 270 : 'W', 90 : 'E', 22.5 :'NNE',
           45 : 'NE', 67.5 : 'ENE', 112.5 : 'ESE', 135 : 'SE', 157.5 : 'SSE',
           202.5 : 'SSW', 225 : 'SW', 247.5 : 'WSW', 292.5 : 'WNW', 315 : 'NW',
           337.5 : 'NNW'}

def deg_to_dir(deg) :
    close_dir = ''
    min_abs = 360
    if deg not in deg_code.keys() :
        for key in deg_code.keys() :
            if abs(key - deg) < min_abs :
                min_abs = abs(key - deg)
                close_dir = deg_code[key]
    else : 
        close_dir = deg_code[deg]
    return close_dir

deg_to_dir(0) # 테스트겸 확인

'N'

# ============연구실================

##### API 결과물 구조
- 1시간당 12개의 아이템 존재.
- 24시간 존재

##### 시간 계산
- 하루 = 12*24 = 288 rows
- 3일 = 864 rows


- 하늘상태(SKY) 코드 : 맑음(1), 구름많음(3), 흐림(4)
- 강수형태(PTY) 코드 : (초단기) 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7) 
                      (단기) 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4) 
- 초단기예보, 단기예보 강수량(RN1, PCP) 범주 및 표시방법(값)